In [ ]:
import numpy as np
import psdr, psdr.demos

# Multiprocessing

When function evaluations are expensive, it becomes advantageous to run multiple evaluations in parallel.  In PSDR we use [dask.distributed](https://docs.dask.org/en/latest/) to orchestrate execution in parallel. The functionality is enabled in the function class by passing a dask client to the `psdr.Function` class when initializing a new function.

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(processes = False, n_workers = 2)
client = Client(cluster)

def f(x):
    import time
    time.sleep(1) # simulates long running time
    return x**2

fun = psdr.Function(f, psdr.BoxDomain(-1,1), dask_client = client)

Like the standard case we can evaluate the function directly, but we also gain a new ability to evaluate the function asyncronously.  This returns a future object.

In [ ]:
res = fun.eval_async([2.])
print(res)

This call is not blocking.  When we are ready to access the result, we call the blocking method

In [ ]:
print(res.result())

If we want to evaluate a number of these simultaneously, we can call `eval_async` with an array of inputs. Now this returns a list of Future objects.

In [ ]:
X = np.linspace(-1,1,11).reshape(-1,1)
res_list = fun.eval_async(X)

for res, x in zip(res_list, X):
    print('x:', x, 'f(x)', res.result())

To implement a poor-man's progress bar, we can use the dask distributed `as_completed` function to return those results that are done and use tqdm to provide a progress bar.  Moreover, we can access the index of the result in the call using the idx property attached to each result object. This allows us to store the values in order.

In [ ]:
from tqdm import tqdm_notebook
from dask.distributed import as_completed
X = np.linspace(-1,1,20).reshape(-1,1)
res_list = fun.eval_async(X)
fX = np.nan*np.zeros( (X.shape[0], 1))
for res in tqdm_notebook(as_completed(res_list), total = len(res_list)):
    print(res.idx)
    fX[res.idx] = res.result()
print(fX)